# CCTV 데이터 시각화 

# 목차

### 1. 라이브러리 임포트
    
### 2. 데이터 적재


### 3. 버블차트 시각화
     3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 
     3.2 2015년, 2020년도 차트 시각화
     3.3 이동궤적 시각화
     3.4 시각화 결과 통합

## 4. 단계구분도 시각화
    4.1 자치구 위치 시각화
    4.2 단계구분도를 반환하는 함수 정의
    4.3 단계구분도 종합


## 1. 라이브러리 임포트

In [38]:
import pandas as pd
import altair as alt

## 2. 데이터 적재

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/cctv_with_pop.csv')
offices = pd.read_csv('https://raw.githubusercontent.com/logistex/vd21/main/offices.csv')
seoul_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/juso/2015/json/seoul_municipalities_topo.json'
seoul = alt.topo_feature(seoul_url, 'seoul_municipalities_geo')

## 3. 버블차트 시각화

### 3.1 회귀선을 포함한 버블차트를 반환하는 함수 정의 


In [40]:
df['CCTV_규모_증가율'] = ((df['CCTV_규모_2020']-df['CCTV_규모_2015'])/df['CCTV_규모_2015'])*100

def bubble(year, color):
    bb = alt.Chart(df).mark_circle(
        opacity=0.5, 
        color=color
    ).transform_calculate(
        cctv_rate=f'datum.CCTV_규모_{year}/datum.인구_규모_{year}', 
    ).encode(
        alt.X(  
            f'인구_규모_{year}:Q',
            scale=alt.Scale(domain=[100000, 700000]), 
            title='인구 규모 (단위: 명)'), 
        alt.Y(
            f'CCTV_규모_{year}:Q',
            scale=alt.Scale(domain=[0, 7000]), 
            title='CCTV 설치 규모 (단위: 대)'), 
        alt.Size(
            f'cctv_rate:Q',
            scale=alt.Scale(range=[10, 1000]), 
            legend=None),   
        alt.Order(
            f'cctv_rate:Q',
            sort='descending'),
        tooltip=['자치구',
                 alt.Tooltip(f'cctv_rate:N', format='.1%', title='CCTV 비율'),
                 alt.Tooltip(f'CCTV_규모_{year}', format=',', title='CCTV 규모'),
                 alt.Tooltip(f'인구_규모_{year}', format=',', title='인구 규모'),
        ],
    )

    reg = bb.transform_regression(
        f'인구_규모_{year}',
        f'CCTV_규모_{year}'
    ).mark_line(
        opacity=0.5,   
        color=color
    )

    return bb + reg     

### 3.2 2015년, 2020년도 차트 시각화

In [41]:
bb_reg_2015 = bubble(2015, 'blue')
bb_reg_2020 = bubble(2020, 'red')
bb_reg_2015 | bb_reg_2020

alt.HConcatChart(...)

### 3.3 이동궤적 시각화

In [42]:
move = alt.Chart(df).mark_line(
).encode(
    alt.X('인구_규모_2015:Q',
          scale=alt.Scale(domain=[100000, 700000]),
          title = '인구 규모 (단위: 명)'
         ),
    alt.X2('인구_규모_2020:Q'),
    
    alt.Y('CCTV_규모_2015:Q',
          scale=alt.Scale(domain=[0, 7000]),
          title = 'CCTV 설치 규모 (단위: 대)'
         ),
    alt.Y2('CCTV_규모_2020:Q'),
    
    alt.Color('CCTV_규모_증가율:Q', legend=None),

    alt.Size('CCTV_규모_증가율:O', legend=None),

    tooltip = [
        '자치구', 
        alt.Tooltip('CCTV_규모_증가율:Q', format='.1%')
    ]

).properties(width=600, height=500,
             title={"text": ["자치구별 인구 대비 CCTV 설치 규모", "(2015년 대 2020년)"],
                    "subtitle": ["-2015년은 푸른 색, 2020년은 빨간색 원으로 표시", 
                                     "-원 크기는 인구 대비 CCTV 비율", 
                                     "-직선 두께와 색상은 2015년 대비 2020년 증가율"]
                    },
)
move

alt.Chart(...)

### 3.4 시각화 결과 통합

In [43]:
layer = alt.layer(bb_reg_2015,
                  bb_reg_2020,
                  move,
                ).properties(
                    width=600, height=500
                )
layer

alt.LayerChart(...)

## 4. 단계구분도 시각화

### 4.1 자치구 위치 시각화

In [44]:
df['CCTV_비율_2015'] = df['CCTV_규모_2015'] / df['인구_규모_2015']
df['CCTV_비율_2020'] = df['CCTV_규모_2020'] / df['인구_규모_2020']

point=alt.layer(
    # alt.Chart(seoul).mark_geoshape(stroke='#aaa', fill='lightgray'
    # ).encode(
    #     tooltip='properties.SIG_KOR_NM:N'
    # ).properties(
    #     width=400, height=300
    # ).project(type='mercator'
    #      ),
    alt.Chart(offices).mark_circle().encode(
        longitude='경도:Q', 
        latitude='위도:Q', 
        size=alt.value(50), 
        tooltip='자치구청'
    ).properties(
        width=400, height=300
    )
)
point

alt.LayerChart(...)

### 4.2 단계구분도를 반환하는 함수 정의

In [45]:
def map(year): 
    return alt.Chart(seoul).mark_geoshape(
        stroke='#aaa', strokeWidth=0.25
    ).transform_lookup(
        lookup='properties.ESRI_PK',
        from_=alt.LookupData(
            data=df, 
            key='no', 
            fields=['자치구', f'인구_규모_{year}', f'CCTV_규모_{year}'])
    ).transform_calculate(
        rate=f'datum.CCTV_규모_{year}/datum.인구_규모_{year}', 
    ).encode(
        color=alt.Color('rate:Q', legend=None),
        tooltip=['자치구:N', 
                 alt.Tooltip('rate:Q', format=',.2%'), 
                 alt.Tooltip(f'인구_규모_{year}:Q', format=','), 
                 alt.Tooltip(f'CCTV_규모_{year}:Q', format=',')]
    ).properties(
         title={
             'text': ['CCTV 설치 비율'], 
             'subtitle': [f'{year}년']
         }, 
        width=400, height=300
    ).project(
        type='mercator'
    )

### 4.3 단계구분도 종합

In [46]:
(map(2015)+point | map(2020)+point).configure_view(
    stroke=None
)

alt.HConcatChart(...)